In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Bidirectional, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

In [4]:
# Load the data
data = pd.read_csv(r'C:\Users\alosh\OneDrive\Desktop\VSCODE\AttackDetection\Datasets\\20.csv', delimiter=',')

In [5]:
data = data[~data.isin([np.nan, np.inf, -np.inf]).any(axis=1)]

In [6]:
# Separate the features and labels
X = data.drop(' Label', axis=1)
y = data[' Label']

In [7]:
# Standardize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [8]:
# One-hot encode the labels
y = pd.get_dummies(y)

# Reshape X
X = np.expand_dims(X, axis=2)

In [9]:
len(data.index)

2059122

In [10]:
print(X.shape)

(2059122, 20, 1)


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from keras.regularizers import l2

# Create the model
model = Sequential()
model.add(Conv1D(50, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), padding='same', kernel_regularizer=l2(0.01)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(50, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(50, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(50, return_sequences=True)))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(3, activation='softmax'))  # Use softmax for multi-class classification

# Compile the model
model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=1)

# Evaluate the model
model.evaluate(X_val, y_val)

# Save the model
model.save('model.h5')

 1164/51479 [..............................] - ETA: 18:01 - loss: 0.1088 - accuracy: 0.9926

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.preprocessing import LabelBinarizer
from keras.models import load_model

# Load the pre-trained model
loaded_model = load_model('25_Features.h5')

# Assuming y_val_labels is a single-column array of labels (e.g., [0, 1, 2, ...])
label_binarizer = LabelBinarizer()

# Assuming loaded_model.predict returns the predicted probabilities for each class
y_pred = loaded_model.predict(X_val)

# Convert probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_val_labels = np.argmax(label_binarizer.fit_transform(y_val), axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_val_labels, y_pred_labels)
print(f'Accuracy: {accuracy}')

# Calculate precision, recall, and F1 score for each class
precision = precision_score(y_val_labels, y_pred_labels, average=None)
recall = recall_score(y_val_labels, y_pred_labels, average=None)
f1 = f1_score(y_val_labels, y_pred_labels, average=None)
mcc = matthews_corrcoef(y_val_labels, y_pred_labels)

# Print precision, recall, and F1 score for each class
for i in range(len(precision)):
    print(f'Class {i}: Precision={precision[i]}, Recall={recall[i]}, F1={f1[i]}')

# Calculate and print macro-average precision, recall, F1 score, and MCC
macro_precision = precision_score(y_val_labels, y_pred_labels, average='macro')
macro_recall = recall_score(y_val_labels, y_pred_labels, average='macro')
macro_f1 = f1_score(y_val_labels, y_pred_labels, average='macro')
macro_mcc = matthews_corrcoef(y_val_labels, y_pred_labels)
print(f'Macro-average Precision={macro_precision}, Recall={macro_recall}, F1={macro_f1}, MCC={macro_mcc}')

12869/12869 [==============================] - 155s 12ms/step
Accuracy: 0.9996308807137602
Class 0: Precision=0.997857721347023, Recall=0.9995289561144113, F1=0.998692639560727
Class 1: Precision=0.9350282485875706, Recall=0.9949899799599199, F1=0.9640776699029127
Class 2: Precision=0.9999973150902667, Recall=0.9996537656342263, F1=0.9998255108505407
Macro-average Precision=0.9776277616749535, Recall=0.9980575672361858, F1=0.9875319401047268, MCC=0.9978682535430629
